In [11]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DoubleType,TimestampType
import pandas as pd

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('module_05_homework') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/29 15:06:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Question 1: Execute spark.version

In [3]:
spark.version

'3.3.2'

##### Answer 01: '3.3.2'

### Question 2: What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

In [6]:
## read the the csv file into pandas dataframe to get schema

fhv_10_df = pd.read_csv('/home/abdol/data_zoomcamp/05-batch/data/raw/fhv/2019/10/fhv_tripdata_2019_10.csv.gz')

In [7]:
fhv_10_df.dtypes

dispatching_base_num       object
pickup_datetime            object
dropOff_datetime           object
PUlocationID              float64
DOlocationID              float64
SR_Flag                   float64
Affiliated_base_number     object
dtype: object

In [8]:
fhv_10_df.head(5)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2019-10-01 00:23:00,2019-10-01 00:35:00,264.0,264.0,NaN,B00009
1,B00013,2019-10-01 00:11:29,2019-10-01 00:13:22,264.0,264.0,NaN,B00013
2,B00014,2019-10-01 00:11:43,2019-10-01 00:37:20,264.0,264.0,NaN,B00014
3,B00014,2019-10-01 00:56:29,2019-10-01 00:57:47,264.0,264.0,NaN,B00014
4,B00014,2019-10-01 00:23:09,2019-10-01 00:28:27,264.0,264.0,NaN,B00014


In [12]:
schema = StructType([
    
    StructField('dispatching_base_num',StringType(),True),
    StructField('pickup_datetime',TimestampType(),True),
    StructField('dropOff_datetime',TimestampType(),True),
    StructField('PUlocationID',IntegerType(),True),
    StructField('DOlocationID',IntegerType(),True),
    StructField('SR_Flag',StringType(),True),
    StructField('Affiliated_base_number',StringType(),True)
])

In [14]:
fhv_10_df = spark.read \
                 .option("header","true") \
                 .schema(schema) \
                 .csv("/home/abdol/data_zoomcamp/05-batch/data/raw/fhv/2019/10/fhv_tripdata_2019_10.csv.gz")

In [15]:
fhv_10_df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [17]:
fhv_10_df = fhv_10_df.repartition(6)

In [18]:
fhv_10_df.write.parquet('/home/abdol/data_zoomcamp/05-batch/data/parquet/fhv/2019/10/')

In [22]:
!ls ../data/parquet/fhv/2019/10 -lh --block-size=MB

total 40MB
-rw-r--r-- 1 abdol abdol 0MB Feb 29 15:18 _SUCCESS
-rw-r--r-- 1 abdol abdol 7MB Feb 29 15:18 part-00000-7bb16ce4-e57a-45c7-9bee-e08eb293efbf-c000.snappy.parquet
-rw-r--r-- 1 abdol abdol 7MB Feb 29 15:18 part-00001-7bb16ce4-e57a-45c7-9bee-e08eb293efbf-c000.snappy.parquet
-rw-r--r-- 1 abdol abdol 7MB Feb 29 15:18 part-00002-7bb16ce4-e57a-45c7-9bee-e08eb293efbf-c000.snappy.parquet
-rw-r--r-- 1 abdol abdol 7MB Feb 29 15:18 part-00003-7bb16ce4-e57a-45c7-9bee-e08eb293efbf-c000.snappy.parquet
-rw-r--r-- 1 abdol abdol 7MB Feb 29 15:18 part-00004-7bb16ce4-e57a-45c7-9bee-e08eb293efbf-c000.snappy.parquet
-rw-r--r-- 1 abdol abdol 7MB Feb 29 15:18 part-00005-7bb16ce4-e57a-45c7-9bee-e08eb293efbf-c000.snappy.parquet


#### Answer 02: Average size = 6 MB

#### Question 03: How many taxi trips were there on the 15th of October?

In [23]:
fhv_10_df.registerTempTable('fhv')

/home/abdol/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [28]:
df_trips_15_count = spark.sql("""
                SELECT 
                        COUNT(*) AS COUNT_TRIPS
                FROM 
                        fhv
                WHERE TO_DATE(pickup_datetime) = '2019-10-15'
        """)

In [29]:
print('COUNT OF TRIPS STARTED AT 15/10 IS {}'.format(df_trips_15_count.show()))

+-----------+
|COUNT_TRIPS|
+-----------+
|      62610|
+-----------+

COUNT OF TRIPS STARTED AT 15/10 IS None


#### ANSWER 03: 62,610

#### Question 04: What is the length of the longest trip in the dataset in hours?

In [35]:
longest_trip_df = spark.sql("""
                SELECT  
                        (unix_timestamp(dropOff_datetime) - unix_timestamp(pickup_datetime)) / 3600 AS hours_difference
                FROM fhv
                ORDER BY hours_difference DESC 
                LIMIT 1
            """)

In [36]:
print('LONGEST TRIP IN HOURS {}'.format(longest_trip_df.show()))

+----------------+
|hours_difference|
+----------------+
|        631152.5|
+----------------+

LONGEST TRIP IN HOURS None


#### ANSWER 04: 631,152.50 Hours

#### Question 05: Spark’s User Interface which shows the application's dashboard runs on which local port?

#### Answer 05: port 4040

#### Question 06:  Least frequent pickup location zone

In [39]:
taxi_zone_schema = StructType([
            StructField('LocationID',IntegerType(),True),
            StructField('Borough',StringType(),True),
            StructField('Zone',StringType(),True),
            StructField('service_zone',StringType(),True)
])

In [40]:
taxi_zone_lkp_df = spark.read \
                        .option("header","true") \
                        .schema(taxi_zone_schema) \
                        .csv("/home/abdol/data_zoomcamp/05-batch/data/lookup/taxi_zone_lookup.csv")

In [41]:
taxi_zone_lkp_df.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [43]:
taxi_zone_lkp_df.createOrReplaceTempView('zone_lkp')

In [47]:
least_frequent_zone_df = spark.sql("""

        SELECT 
                fhv.PUlocationID,
                zone_lkp.Zone,
                COUNT(*) AS COUNT_TRIPS
        FROM 
            fhv 
            LEFT JOIN
            zone_lkp 
            ON fhv.PUlocationID = zone_lkp.LocationID
        GROUP BY 1,2
        ORDER BY COUNT_TRIPS ASC
        LIMIT 1
             """)

In [50]:
print('LEAST FREQUENT ZONE IS {} '.format(least_frequent_zone_df.select('Zone').show()))

+-----------+
|       Zone|
+-----------+
|Jamaica Bay|
+-----------+

LEAST FREQUENT ZONE IS None 


#### Answer 06: Jamaica Bay

In [51]:
spark.stop()